# Setting

In [1]:
# 코드나 내용에서 변경있으면 바로 업데이트해서 사용할 수 있게 하는 것
%load_ext autoreload
%autoreload 2

In [2]:
from google.colab import drive
drive.mount('/content/drive')
import os
os.chdir('/content/drive/MyDrive')

Mounted at /content/drive


In [3]:
# 현재 활동하고 있는 경로 
os.getcwd()

'/content/drive/MyDrive'

In [4]:
# 간혹 경로문제가 생길까봐 검색할 때 사용할 기본 경로
# sys.path 는 디렉터리의 경로들이 기록된 문자열 리스트
#이 리스트에 경로를 추가하면 해당 경로에 있는 파이썬 파일을 import 문으로 불러올 수 있음
import sys

sys.path.append('/content/drive/MyDrive')

In [5]:
sys.path

['/content',
 '/env/python',
 '/usr/lib/python37.zip',
 '/usr/lib/python3.7',
 '/usr/lib/python3.7/lib-dynload',
 '',
 '/usr/local/lib/python3.7/dist-packages',
 '/usr/lib/python3/dist-packages',
 '/usr/local/lib/python3.7/dist-packages/IPython/extensions',
 '/root/.ipython',
 '/content/drive/MyDrive']

In [6]:
# 우리가 사용하는 GPU 상태
!nvidia-smi

Thu Oct  6 08:15:12 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   60C    P8    10W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [7]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 4.9 MB 28.5 MB/s 
     |████████████████████████████████| 6.6 MB 55.7 MB/s 
     |████████████████████████████████| 163 kB 74.3 MB/s 


In [8]:
!pip install pyaml

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [9]:
os.getcwd()

'/content/drive/MyDrive'

# Package


In [10]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import DataLoader

import json
import yaml
import random
import pickle
import numpy as np
from tqdm import tqdm
import datetime

# from transformers import *
# from transformers import GPT2LMHeadModel


# Data

In [11]:
with open("criminal.txt", 'r') as f:
  raw = f.readlines()
raw_two = raw[0].split('   ')
raw_two = list(map(lambda x: x[:-3], raw_two))
raw_two = [i for i in raw_two if len(i)>3]

In [12]:
raw_two

[' 걱정마! 이길거야.교도소장김선생  예전에 말이야  일본 경시청에서 범죄자 놈들 500명을 조사했어.찬밥  말라비틀어진 김치  눅진 김. 딱 세 가지.배꼽노사범님  은퇴하셨으면 놀지 말고 후학 양성하세요.배꼽왕사범님  불만 있으면 목숨 걸고 한판 두시던가.그때  검지와 중지를 꼬며 중국식 손숫자 10표시를 하는 량량.선수야  새끼야. 수금은 했어?선수야  당장 텨오지 않고 뭐해?!태석  내기는 이제부터야.꽁수아  곧 갈 테니 편의점 앞에서 기다리쇼.아  니미... 외벽을 잡고 아등바등 기어오르려는 순간꽁수누  누구? 이봐  형씨. 사람 잘못 찾은 거 아냐?태석알아  몰라?꽁수레  레크레이션!!!꽁수허  운 좋구만.꽁수어이  큰돌.  그러지 말고 우리 둘이 해먹자. 응?어이  지자쓰. 적당히 좀 드시지?꽁수어이  지자쓰. 이쯤 왔으면 결정하지?살수이사범님  오늘은 왜 또 실수하셨습니까?호구사장어이  친구! 패 쌓으러 왔으면 이리 와.호구사장이야  인물 좋네. 잘 왔어. 내가 위스키 한 잔 쏠게.야  근데 쓰러진 놈 옆에서 “이봐  마작 하려는데 한 사람 부족해!”했더니 벌떡 일어나는 거야.  우리 무슨 얘기 하고 있었지?호구사장아차  학교. 암튼 중요한 건 우리가 이렇게 만났다는 거야.꽁수아  세네! 세!꽁수허목  싸구려 써서 끊기는 건 아니겠지?꽁수지자쓰  내가 엄청난 은둔고수란 생각 안 해봤어?허목수  좌하귀 화점.선수사장님  괜찮으면 한판 두시겠습니까?꽁수어이구  송프로 공사가 다망하셔? 좋은 데 있으면 같이 좀 다니자구.꽁수어이구  쿨하십니다.꽁수아  미안합니다. 제가 빠른 거는 좀 거부감이 있어서.태석친목인가요  내기인가요?아차  거기 냉동고 좀 꺼주고. 리모컨은 캐비닛 안에.살수  앉아.왕사범야  새끼들아! 이거 안 놔?!',
 '판돈 20억 다 걸지. 대신 그쪽은 왕사범 혀를 걸었으면 하는데왕사범오  오사장. 우리끼리 이러면 안 돼.왕사범함  함정이야. 오사장.',
 '바둑판 뒤집어봐왕사범오  오사장. 내가 잡아올게.',
 '재앙은 입으로부터 나

# 테스트

In [13]:
from transformers import EncoderDecoderModel

In [14]:
bert2bert = EncoderDecoderModel.from_pretrained("./ckpt/YJ/checkpoint-3000")

OSError: ignored

In [ ]:
from transformers import PreTrainedTokenizerFast

from transformers import AutoModel, AutoTokenizer

In [ ]:
tokenizer = AutoTokenizer.from_pretrained("klue/bert-base")

In [ ]:
vocab = tokenizer.vocab

for k,v in vocab.items():
  if v <= 50:
    print(k, v)

In [ ]:
len(tokenizer.vocab)

# Run

In [ ]:
# sample = "나는 딴 돈에 반만 가져가요"
# sample = "첫판부터? 삼촌 오늘 저 말리지 마세요."
# sample = "아예 둘 다 잘라주십쇼"
# sample = "그년을 이렇게 만나나"
# sample = "사장님 그만 하시죠"
# sample = "내가 건달생활을 열일곱에 시작했다."
# sample = "건물이라도 담보를 잡든가요."
# sample = "어디 실실 쪼개 이 십새끼가?"
# sample="우린 전화 열통이면 전국 다 수배돼."
# sample = "우리 밥먹으러 가자."
# sample = "수업 끝나고 옥상으로 와."
# sample = "과제는 다음주까지 제출하세요."
# sample = "곧 갈 테니 편의점 앞에서 기다리쇼."
# sample = "밑장을 빼면 소리가 달라!"
sample = "도박의 꽃이 누군줄 아냐?"

In [ ]:
# sample = "첫판부터? 삼촌 오늘 저 말리지 마세요."

In [ ]:
sample_inputs = tokenizer(sample, padding="max_length", truncation=True, max_length=32)

In [ ]:
inputs = tokenizer([sample])
inputs

In [ ]:
for i in inputs.input_ids[0]:
  token = tokenizer._convert_id_to_token(i)
  print(i, token)
  

In [ ]:
masked = []
inputs.input_ids[0][-4:-2] = [4]*2
for i in inputs.input_ids[0]:
  token = tokenizer._convert_id_to_token(i)
  masked.append(token)
masked

In [ ]:
tokenizer.convert_tokens_to_string(masked)

In [ ]:
# input_ids = torch.LongTensor([i[1:] for i in inputs.input_ids]).cuda()
input_ids = torch.LongTensor(inputs.input_ids)
input_ids

In [ ]:
attention_mask = torch.LongTensor(
    # [i[1:] for i in inputs['attention_mask']]
    inputs['attention_mask']
    )
attention_mask

In [ ]:
token_type_ids = torch.LongTensor(
    #  [i[1:] for i in inputs['token_type_ids']]
     inputs['token_type_ids']
    )
token_type_ids

In [ ]:
out = bert2bert(input_ids, attention_mask, token_type_ids)

## 후처리

In [ ]:
a = out.logits.squeeze(0)
a

In [ ]:
a.size()

In [ ]:
a[:, :50] = -float("inf")
a

In [ ]:
vocab_act ={str(k):0 for k in tokenizer.vocab.values()}

In [ ]:
for line in raw_two[:2]:
  token_ids = tokenizer.batch_encode_plus([line]).input_ids[0][1:-1]
  for toekn_id in token_ids:
    vocab_act[str(toekn_id)] += 1

In [ ]:
# target_token = [int(i) for i in sorted(vocab_act, key=vocab_act.get)[:3000]]
target_token = [int(i) for i in vocab_act.keys() if vocab_act[i]<1]

In [ ]:
# sorted(target_token)

In [ ]:
# [tokenizer._convert_id_to_token(i) for i in range(len(vocab)) if i not in target_token]

In [ ]:
len(target_token)

In [ ]:
# target_tokens = []
# for i in target_token:
#     target_tokens.append(tokenizer._convert_id_to_token(i))
# # target_tokens

In [ ]:
a[:, target_token] = -float("inf")

In [ ]:
# a[:, 5971] = -float("inf")

In [ ]:
torch.isfinite(a).sum()

In [ ]:
count = 0
result = []
for i in range(len(a)):
    if input_ids[0][i] ==4:
        i_max = a[i].argmax()
        count += 1
        result.append(i_max.item())        
        a[i+1:, i_max] = -float("inf")
    else:
      if input_ids[0][i] in [2, 3]:
        continue
      else:
        result.append(input_ids[0][i].item())
result

In [ ]:
tokens = []
for i in result:
    tokens.append(tokenizer._convert_id_to_token(i))
tokens

In [ ]:
tokenizer.convert_tokens_to_string(tokens)

In [ ]:
tokens = []
for i in range(len(a)):
    tokens.append(tokenizer._convert_id_to_token(a[i].argmax().item()))
tokens